In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt


In [ ]:
loaded = np.load("../data/processed/data32.npz")

train_images = loaded["train_images"]
test_images = loaded["test_images"]
train_labels = loaded["train_labels"]
test_labels = loaded["test_labels"]

SIZE = train_images[0].shape[0]

In [ ]:


plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    idx = np.random.choice(len(train_images))
    plt.imshow(train_images[idx], cmap = "gray")
    if(train_labels[idx] == 1):
        text = "Smiling"
    else:
        text = "Not Smiling"
    plt.xlabel(text, color = "white")
plt.show()

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(SIZE, (3, 3), activation='relu', input_shape=(SIZE, SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(SIZE * 2, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(SIZE, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(SIZE, activation='relu'))
model.add(layers.Dense(1))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])



In [ ]:
history = model.fit(train_images, train_labels, epochs=25, 
                    validation_data=(test_images, test_labels))

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1])
plt.legend(loc='lower right')


In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    idx = np.random.choice(len(test_images))
    plt.imshow(test_images[idx], cmap = "gray")
    text = "{:.2f}".format(tf.math.sigmoid(model.predict(np.array( [test_images[idx],]))[0][0]))
    plt.xlabel(text, color = "white")
plt.show()

In [ ]:
model.save("model_" + str(SIZE))